# 📊 Entraînement des modèles ML pour NutriProfil
Ce notebook regroupe les entraînements et sauvegardes de tous les modèles :
- Régression Lasso (prévision consommation)
- Régression linéaire (charge glycémique)
- RandomForestClassifier (risques : diabète, obésité, MCV, cancer colorectal)

Les `.pkl` seront sauvegardés dans `streamlit/`.


In [1]:
# 📦 Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from pathlib import Path
import os
import requests

# Dossier de destination
destination_dir = "streamlit"
os.makedirs(destination_dir, exist_ok=True)

# URLs brutes de GitHub
base_url = "https://raw.githubusercontent.com/stevens75010/nutriprofil/main/streamlit/"
model_files = [
    "diabete_risque_randomforest.pkl",
    "obesite_risque_randomforest.pkl",
    "mcv_risque_randomforest.pkl",
    "cancercolorectal_risque_randomforest.pkl"
]

# Téléchargement automatique
for filename in model_files:
    file_url = base_url + filename
    local_path = os.path.join(destination_dir, filename)
    
    if not os.path.exists(local_path):
        print(f"📥 Téléchargement de {filename}...")
        r = requests.get(file_url)
        if r.status_code == 200:
            with open(local_path, "wb") as f:
                f.write(r.content)
            print(f"✅ Enregistré : {local_path}")
        else:
            print(f"❌ Échec du téléchargement : {file_url}")
    else:
        print(f"🔁 Déjà présent : {local_path}")


🔁 Déjà présent : streamlit\diabete_risque_randomforest.pkl
🔁 Déjà présent : streamlit\obesite_risque_randomforest.pkl
🔁 Déjà présent : streamlit\mcv_risque_randomforest.pkl
🔁 Déjà présent : streamlit\cancercolorectal_risque_randomforest.pkl


## 🔢 1. Modèle de prévision de consommation alimentaire (Lasso)

In [2]:
# Chargement des données conso
df_conso = pd.read_csv('streamlit/data/conso-menages-2024.csv')
cols = [str(y) for y in range(2010, 2025)]
for col in cols:
    df_conso[col] = df_conso[col].astype(str).str.replace(r'\s+', '', regex=True).str.replace(',', '.').astype(float)

X = df_conso[cols[:-1]]
y = df_conso[cols[-1]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_conso = Lasso()
model_conso.fit(X_train, y_train)
print("R²:", r2_score(y_test, model_conso.predict(X_test)))
joblib.dump(model_conso, 'streamlit/model_conso.pkl')

R²: 0.9998725252973537


['streamlit/model_conso.pkl']

## 🍚 2. Modèle charge glycémique (régression linéaire)

In [3]:
# Chargement données aliments
df_alim = pd.read_csv('streamlit/data/dataframe_complet_rempli_proteines.csv')
df_alim['Famille_regroupee'] = df_alim['Famille_regroupee'].replace('Fruits et lÃ©gumes', 'Fruits et légumes')
df_alim = pd.concat([df_alim, pd.get_dummies(df_alim['Famille_regroupee'], prefix='Famille', drop_first=True)], axis=1)

X = df_alim.drop(columns=['Aliment', 'CG', 'Famille_regroupee','IG'])
y = df_alim['CG']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_cg = LinearRegression()
model_cg.fit(X_train, y_train)
print("R² CG:", r2_score(y_test, model_cg.predict(X_test)))
joblib.dump(model_cg, 'streamlit/model_charge_glycémique.pkl')

R² CG: 0.8907643393336107


['streamlit/model_charge_glycémique.pkl']

## 🧠 3. Modèles de risques santé (RandomForestClassifier)

In [4]:
# Réutilisation df_alim pour entraîner 4 modèles de classification binaire
features = df_alim.drop(columns=['Aliment', 'Famille_regroupee', 'IG', 'CG'])
targets = ['diabete', 'obesite', 'mcv', 'cancercolorectal']

# Simuler des colonnes cibles aléatoires pour l'exemple (remplace par ton vrai DataFrame si dispo)
for t in targets:
    df_alim[t] = np.random.randint(0, 2, size=len(df_alim))

for t in targets:
    X_train, X_test, y_train, y_test = train_test_split(features, df_alim[t], test_size=0.2, random_state=42)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    joblib.dump(model, f'streamlit/{t}_risque_randomforest.pkl')
    print(f"✅ {t} — R² approx :", model.score(X_test, y_test))

✅ diabete — R² approx : 0.5193704600484261
✅ obesite — R² approx : 0.4648910411622276
✅ mcv — R² approx : 0.48426150121065376
✅ cancercolorectal — R² approx : 0.49878934624697335


In [5]:
# 🔍 Vérification des 6 fichiers modèles
import os

required_models = [
    "model_charge_glycémique.pkl",
    "model_conso.pkl",
    "diabete_risque_randomforest.pkl",
    "obesite_risque_randomforest.pkl",
    "mcv_risque_randomforest.pkl",
    "cancercolorectal_risque_randomforest.pkl"
]

missing = []
for model in required_models:
    path = os.path.join("streamlit", model)
    if os.path.exists(path):
        print(f"✅ Présent : {model}")
    else:
        print(f"❌ Manquant : {model}")
        missing.append(model)

if not missing:
    print("\n🎉 Tous les modèles sont bien présents !")
else:
    print(f"\n⚠️ Modèles manquants : {missing}")


✅ Présent : model_charge_glycémique.pkl
✅ Présent : model_conso.pkl
✅ Présent : diabete_risque_randomforest.pkl
✅ Présent : obesite_risque_randomforest.pkl
✅ Présent : mcv_risque_randomforest.pkl
✅ Présent : cancercolorectal_risque_randomforest.pkl

🎉 Tous les modèles sont bien présents !
